In [104]:
import pandas as pd
import numpy as np

In [105]:
col_types = {'Reservation Creation Date (DD-MM-YYYY)':str, }
df = pd.read_excel('./RequiredData.xlsx', sheet_name="Reservation List")
df.columns

Index(['Reservation ID', 'Reservation Creation Date (MM-DD-YYYY)',
       'Reservation Check-in Date', 'Reservation Check-out Date ',
       'Reservation Total Price ', '#Rooms', 'Cancellation Date',
       'Type\nallotment (1),  sales allotment (2), direct hotel (3)or direct agent(4)'],
      dtype='object')

In [106]:
df.columns = ['rid', 'cr_date', 'ci_date', 'co_date', 'tot_price', 'num_rooms', 'cancel_date', 'res_type']

In [107]:
df.head(10)

,rid,cr_date,ci_date,co_date,tot_price,num_rooms,cancel_date,res_type
0,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1
1,2,2023-01-01,2023-05-04,2023-05-09,5000,5,NaT,1
2,3,2023-01-01,2023-08-09,2023-08-10,200,5,2023-01-03,1
3,4,2023-01-02,2023-09-07,2023-09-16,7200,4,2023-01-03,1
4,5,2023-01-02,2023-01-09,2023-01-12,81400,20,NaT,2
5,6,2023-01-03,2023-04-21,2023-04-27,74243,55,NaT,1


In [108]:
def convert_to_datetime(df, inplace=True):
    if not inplace:
        df = df.copy()
    for i in range(len(df.index)):
        for j in range(len(df.columns)):
            if 'date' in df.columns[j]:
                try:
                    df.iloc[i, j] = pd.to_datetime(df.iloc[i, j])
                except:
                    df.iloc[i, j] = pd.to_datetime(df.iloc[i, j])
    if inplace:
        return
    else:
        return df

In [109]:
# def convert_to_datetime(df, inplace=True):
#     if not inplace:
#         df = df.copy()
#     for i in range(len(df.index)):
#         for j in range(len(df.columns)):
#             if 'date' in df.columns[j]:
#                 done = False
#                 k = 0
#                 possible_formats = ['%d/%m/%Y', '%d/%m/%Y HH:MM:SS', '%Y/%d/%m', '%Y/%d/%m HH:MM:SS', '%Y/%m/%d', '%Y/%m/%d HH:MM:SS']
#                 while not done:
#                     try:
#                         print(df.iloc[i,j])
#                         df.iloc[i, j] = pd.to_datetime(df.iloc[i, j], format=possible_formats[k])
#                         print(df.iloc[i,j])
#                         done = True
#                     except:
#                         k += 1
#                         if k == len(possible_formats):
#                             break
#     if inplace:
#         return
#     else:
#         return df

In [110]:
convert_to_datetime(df, inplace=True)

In [111]:
df.sort_values('cr_date', inplace=True)

In [112]:
df.head(10)

,rid,cr_date,ci_date,co_date,tot_price,num_rooms,cancel_date,res_type
0,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1
1,2,2023-01-01,2023-05-04,2023-05-09,5000,5,NaT,1
2,3,2023-01-01,2023-08-09,2023-08-10,200,5,2023-01-03,1
3,4,2023-01-02,2023-09-07,2023-09-16,7200,4,2023-01-03,1
4,5,2023-01-02,2023-01-09,2023-01-12,81400,20,NaT,2
5,6,2023-01-03,2023-04-21,2023-04-27,74243,55,NaT,1


In [113]:
def get_cancel_dates(df):
    cancel_dates_to_values = {}
    for index, row in df.iterrows():
        
        cancel_dt = row['cancel_date']
        if cancel_dt is pd.NaT:
            continue
        if cancel_dt not in cancel_dates_to_values.keys():
            cancel_dates_to_values[cancel_dt] = np.array([0, 0]) # res_total_price and numRooms
        cancel_dates_to_values[cancel_dt] += np.array([row['tot_price'], row['num_rooms']])

    return cancel_dates_to_values

In [114]:
def get_cf_df_from_res(df, res_type='all', date_as_index=False):
    df = df.copy()
    if res_type != 'all':
        df = df[df['res_type']==int(res_type)]
    df = df.groupby('cr_date').sum(numeric_only=True)
    df = df[['tot_price', 'num_rooms']]
    if not date_as_index:
        df = df.reset_index()
    return df

In [115]:
def get_cf_df_from_pay(df, date_as_index=False):
    df = df.copy()
    df = df.groupby('pay_date').sum(numeric_only=True)
    df = df[['pay_amount']]
    if not date_as_index:
        df = df.reset_index()
    return df

In [116]:
cf_df_before_cancel = get_cf_df_from_res(df, res_type=1, date_as_index=False)

In [117]:
cf_df_before_cancel

,cr_date,tot_price,num_rooms
0,2023-01-01,29200,20
1,2023-01-02,7200,4
2,2023-01-03,74243,55


In [118]:
# "values" here refer to an np.array() of two values: reservation total price & #rooms
cancel_dates_to_values = get_cancel_dates(df)

# Define the function to subtract values based on dictionary's key-value mapping
def subtract_values_from_cf_res(row):
    cur_date = row['cr_date']
    subtract_array = cancel_dates_to_values.get(cur_date, np.array([0, 0]))
    row[['tot_price', 'num_rooms']] -= subtract_array
    return row

def subtract_values_from_cf_pay(row):
    cur_date = row['cr_date']
    subtract_array = cancel_dates_to_values.get(cur_date, np.array([0, 0]))
    row[['pay_amount']] -= subtract_array
    return row


# Apply the function to subtract values based on dictionary's key-value mapping
cf_df = cf_df_before_cancel.apply(subtract_values_from_cf_res, axis=1)
cf_df

,cr_date,tot_price,num_rooms
0,2023-01-01,29200,20
1,2023-01-02,7200,4
2,2023-01-03,66843,46
